In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *


spark = SparkSession \
    .builder \
    .appName("word-count") \
#     .master("local[3]") \
#     .config("spark.streaming.stopGracefullyOnShutdown", "true") \
#     .config("spark.sql.shuffle.partitions", 3) \
    .getOrCreate()

In [24]:
lines_df = spark.readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", "9999") \
    .load()

lines_df.printSchema()

root
 |-- value: string (nullable = true)



In [25]:
words_df = lines_df.select(explode(split("value", " ")).alias("word"))
# words_df = lines_df.select(expr("explode(split(value,' ')) as word"))
counts_df = words_df.groupBy("word").count()

In [28]:
word_count_query = counts_df.writeStream \
    .format("console") \
    .outputMode("complete") \
    .option("checkpointLocation", "chk-point-dir") \
    .start()


KeyboardInterrupt: 

In [ ]:
word_count_query.awaitTermination()